In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from IPython.display import clear_output
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt
import pandas as pd

import quick_pp.las_handler as las

In [ ]:
# Loading a dataframe

root = Tk()
file_objects = filedialog.askopenfiles(title='Choose well Log ASCII Standard (LAS) files to be combined',
                                        filetype=(('LAS Files', '*.LAS *.las'), ('All Files', '*.*')),
                                        multiple=True,
                                        mode='rb')
root.destroy()
df, _ = las.read_las_files(file_objects)

In [ ]:
print([c for c in df.columns if 'RT' in c])

In [ ]:
from quick_pp.core_calibration import poroperm_func, fit_poroperm_curve, poroperm_xplot

for rt, rt_data in df.groupby('RT_V2_2023'):
    a, b = fit_poroperm_curve(rt_data['CPORE'], rt_data['CPERM'])
    cpore = np.linspace(0.05, 0.5, 50)
    poroperm_xplot(rt_data['CPORE'], rt_data['CPERM'], a, b, label=f'RT {rt} - a: {a}, b: {b}')
    # plt.show()


In [ ]:
from quick_pp.core_calibration import j_func, fit_j_curve, j_xplot

well_name = 'BA-H010'
well_df = df[df['WELL_NAME'] == well_name]

for well, well_data in well_df.groupby('RT_V3_2023'):
    temp_df = pd.DataFrame()
    for i, data in well_data.iterrows():
        t_df = data[[c for c in test.columns if 'SW[' in c]].reset_index(drop=True).T.reset_index(name='SW')
        # t_df = pd.concat([t_df, data[[c for c in test.columns if 'PC' in c]].reset_index(drop=True).T.reset_index(name='PC')], axis=1)
        # t_df = pd.concat([t_df, data[[c for c in test.columns if 'SWN' in c]].reset_index(drop=True).T.reset_index(name='SWN')], axis=1)
        t_df = pd.concat([t_df, data[[c for c in test.columns if 'J' in c]].reset_index(drop=True).T.reset_index(name='J')], axis=1)
        t_df.drop(columns='index', inplace=True)
        temp_df = pd.concat([temp_df, t_df])
    temp_df.dropna(inplace=True)
    temp_df.reset_index(drop=True, inplace=True)
    a, b = fit_j_curve(temp_df['SW'], temp_df['J'])
    j_xplot(temp_df['SW'], temp_df['J'], a, b, label=f"RT {data['RT_V3_2023']} - a: {a}, b: {b}")
    plt.show()


In [ ]:
# Need HAFWL to plot 
df.plot(x='BVW', y='DEPTH_SHIFTED', kind='scatter', logy=False, xlim=(0, 0.5))

In [ ]:
# df[[c for c in df.columns if 'PC' in c]].describe()

In [ ]:
# for i in range(8):
#     df.plot(x=f'SW[{i}]', y=f'PC[{i}]', kind='scatter', logy=False, xlim=(0, 1))
# plt.show()